# Unseen Test Data Preparation

Point Cloud NN requires data in the following scheme:

dataset

--- class1  

    --- train  
        * file.off
        * file.off
    
    --- test 
        * file.off
        * file.off
    

--- class2  

    --- train 
        * file.off
        * file.off
    
    --- test  
        * file.off
        * file.off

In [1]:
def read_csv(READ_PATH):
    """
    Read CSV at specified Path
    """
    return pd.read_csv(READ_PATH)


def identify_groups(df):
    """
    1. Tag groups that have only noise [0]
    2. Tag groups that have both noise and hits [0,1]
    3. Separate noise groups and hits+noise groups
    """
    # Label groups by noise and and hits
    df_count_label_type = pd.DataFrame(df.groupby('group')['label'].unique()).reset_index()
    
    # Obtain groups with only hits
    df_noise = df_count_label_type.loc[
        np.array(list(map(len, df_count_label_type.label.values))) == 1]
    
    # Obtain groups with noise && hits
    df_mixed = df_count_label_type.loc[
        np.array(list(map(len, df_count_label_type.label.values))) > 1]
    
    return df_noise, df_mixed


def generate_dfs(df, df_noise, df_mixed):
    """
    Obtain full dataframe based on identified noise
    only groups and mixed groups
    """
    df_noise = df[df.group.isin(df_noise.group)]
    df_mixed = df[df.group.isin(df_mixed.group)]
    
    return df_noise, df_mixed


def print_df_stats(df_noise, df_mixed):
    """
    General Stats on mixed and noise groups
    """
    noise_stats = df_noise.groupby('group')['label']\
                    .count()\
                    .sort_values(ascending=False)\
                    .describe()
    mixed_stats = df_mixed.groupby('group')['label']\
                    .count()\
                    .sort_values(ascending=False)\
                    .describe()
    
    neg, pos = np.bincount(df_mixed['label'])
    total = neg + pos
    
    hits = df_mixed[df_mixed.label == 1].groupby(['group', 'label'])['label']
    noise = df_mixed[df_mixed.label == 0].groupby(['group', 'label'])['label']
    
    hits_count = hits.count().sort_values(ascending=False)[:SIZE]
    noise_count = noise.count().sort_values(ascending=False)[:SIZE]
    
    hits_to_noise = hits_count.droplevel(level='label')/noise_count.droplevel(level='label')
    
    class_imbalance = df_mixed.groupby(['group', 'label'])['label'].count()[:20]
    
        
    print("NOISE STATS: \n{}\n".format(noise_stats))
    print("MIXED STATS: \n{}\n".format(mixed_stats))
    
    print("Mixed Groups: \n")
    print("Examples:\n Total: {}\n Positive: {} ({:.2f}% of total)\n".format(total,
                                                                             pos, 100 * pos / total))
    
    print("Example of class imbalance in mixed groups: \n ".format(class_imbalance.head()))
           
    print("Hits Only (Within mixed groups):\n")
    print("The largest hits for a group: {}\n".format(hits_count.max()))
    print("The smallest hits for a group: {}\n".format(hits_count.min()))
    print("The mean hits for a group: {}\n".format(hits_count.mean()))
                                                                                                                                    
    print("Noise Only (Within mixed groups):\n") 
    print("The largest noise for a group: {}\n".format(noise_count.max()))
    print("The smallest noise for a group: {}\n".format(noise_count.min())) 
    print("The mean noise for a group: {}\n".format(noise_count.mean()))
    print("Hits-Noise Ratio:\n") 
    print("The highest ratio: {:.2f}% ({})\n".format(hits_to_noise.max()*100,
                                                    hits_to_noise.max()))
    print("The smallest ratio: {:.2f}% ({}) \n".format(hits_to_noise.min()*100,
                                                       hits_to_noise.min())) 
    print("The mean ratio: {:.2f}% ({})\n".format(hits_to_noise.mean()*100, 
                                                  hits_to_noise.mean())) 

def sort_mixed_groups(df):
    """
    Within each mixed groups (hits + noise), we need to sort
    groups such that those with highest hits are selected first
    
    DataFrame --> List(int)
    """
    
    # group by groups and labels and associated counts for each label
    grouped_df = pd.DataFrame(df.groupby(['group', 'label'])['label'].count())
    grouped_df = grouped_df.rename(columns={'label':'count'})
    
    # sort groups based on highest occurance of hits
    grouped_sorted_df = grouped_df.sort_values(grouped_df.columns.tolist())\
                            .sort_index(level=1, ascending=False, sort_remaining=False)\
                            .reset_index()
    
    # Obtain list of groups with highest hits based on sorted order
    sorted_groups_list = pd.DataFrame(grouped_sorted_df.group)
    
    # Drop duplicate groups
    sorted_groups_list = sorted_groups_list.drop_duplicates()
    
    # Count the occurances of groups (should only occur once)
    sorted_groups_list['g'] = sorted_groups_list.groupby('group').cumcount()
    
    # Make copy of dataframe
    copy_df = df
    
    # Save original index positions as a column
    copy_df_indices = copy_df.reset_index()
    
    # Make a count of occurances for each group
    copy_df_indices['group_count'] = copy_df_indices.groupby('group').cumcount() 
    
    # Merge the list of groups with the partial df to obtain corresponding full dataframe
    copy_df = sorted_groups_list.merge(copy_df_indices)\
                                .set_index('index')\
                                .rename_axis(None)\
                                .drop(['group_count', 'g'], axis=1)
    
    # For each group, sort by labels within each group starting from 1 till 0
    df = copy_df.groupby(['group'], sort=False)\
                 .apply(lambda x: x.sort_values(['label'], ascending=False))\
                 .reset_index(drop=True)
    
    return df    

 
    
def save_unsampled_noise(df_noise, WRITE_NOISE):
    """
    Save unsampled noise data
    """
    list_noise_groups = df_noise.groupby('group')['label'].count().sort_values(ascending=False)[SIZE:SIZE + 10]

    list_noise_groups = list(list_noise_groups.index)
    
    for idx in list_noise_groups:
        file_name = "group_"+str(idx)+".xyz"
        np.savetxt(WRITE_NOISE + file_name,
                   df_noise[df_noise.group == idx][['pos_x', 'pos_y', 'time']].values)
    
    print("All {0} files saved successfully in {1}!".format(len(list_noise_groups), WRITE_NOISE))
    


def save_unsampled_mixed(df_mixed, WRITE_MIXED):
    """
    Save unsampled mixed groups
    """
    
    sorted_mixed_df = sort_mixed_groups(df_mixed)
    list_mixed_groups = sorted_mixed_df.group.unique()[SIZE:SIZE + 10]
    
    for idx in list_mixed_groups:
        file_name = "group_"+str(idx)+".xyz" 
        np.savetxt(WRITE_MIXED + file_name,
                   df_mixed[df_mixed.group == idx][['pos_x', 'pos_y', 'time']].values)
    
    print("All {0} files saved successfully in {1}!".format(len(list_mixed_groups),
                                                            WRITE_MIXED))
    
    
def plot(df_mixed):
    """
    Generate plots of full data in mixed groups
    """
    pos_df = pd.DataFrame(df_mixed[df_mixed.label == 1], columns = df_mixed.columns)
    neg_df = pd.DataFrame(df_mixed[df_mixed.label == 0], columns = df_mixed.columns)
    sns.jointplot(pos_df['pos_z'], pos_df['time'],
                  kind='hex')
    plt.suptitle("Positive distribution (Hits) for pos_z vs time")
    sns.jointplot(neg_df['pos_z'], neg_df['time'],
                  kind='hex')
    _ = plt.suptitle("Negative distribution (Noise) for pos_z vs time")
    
    plt.savefig("../../assets/Distributon of points in Mixed Groups (posz-time)")
    

def sampled_plot(sampled_mixed):
    """
    Generate plots of equally sampled points in mixed groups
    """
    list_mixed_groups = df_mixed.groupby('group')['label'].count().sort_values(ascending=False)[:SIZE]
    data_subset = sampled_mixed.loc[sampled_mixed['group'].isin(list_mixed_groups)]
    pos_df = data_subset[data_subset.label == 1]
    neg_df = data_subset[data_subset.label == 0]
    
    sns.jointplot(pos_df['pos_z'], pos_df['time'],
                  kind='hex')
    plt.suptitle("EQUAL SAMPLING: Positive distribution (Hits) for pos_z vs time")
    
    sns.jointplot(neg_df['pos_z'], neg_df['time'],
                  kind='hex')
    _ = plt.suptitle("EQUAL SAMPLING: Negative distribution (Noise) for pos_z vs time")

    plt.savefig("../../assets/Distributon of Equally Sampled points in Mixed Groups (posz-time)")

    
def main():
    df = read_csv(READ_PATH)
    df_noise, df_mixed = identify_groups(df)
    df_noise, df_mixed = generate_dfs(df, df_noise, df_mixed)
    
    save_unsampled_noise(df_noise, WRITE_NOISE)
    save_unsampled_mixed(df_mixed, WRITE_MIXED)
    
    to_plot = input("Generate plots? (y/n) \n")
    if to_plot == "y":
        plot(df_mixed)
        sampled_plot(sampled_mixed)
    else:
        return df_noise, df_mixed   

In [29]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

READ_PATH = "../../../data/simplified_data.csv"
WRITE_NOISE = "../../../data/unsampled/test/points/noise/"
WRITE_MIXED = "../../../data/unsampled/test/points/mixed/1063"

SIZE = 200

# df_noise, df_mixed= main()

In [30]:
df = read_csv(READ_PATH)
# df_noise, df_mixed = identify_groups(df)

In [4]:
df_noise, df_mixed = generate_dfs(df, df_noise, df_mixed)

In [6]:
sorted_mixed_df = sort_mixed_groups(df_mixed)

KeyboardInterrupt: 

In [14]:
sorted_mixed_df[200:]

,group,pos_x,pos_y,pos_z,time,label
200,615,40.287,-3.331,83.300,9232435.0,1
201,615,59.248,31.905,73.931,9232437.0,1
202,615,88.367,12.534,83.389,9232438.0,1
203,615,48.127,-24.132,65.231,9232438.0,1
204,615,40.143,-3.414,47.411,9232424.0,1
...,...,...,...,...,...,...
27485991,2816,-8.755,13.555,186.931,42245023.0,0
27485992,2816,11.668,-57.877,121.841,42245023.0,0
27485993,2816,22.778,-113.760,187.211,42245022.0,0
27485994,2816,59.035,66.337,122.011,42245010.0,0


In [11]:
list_mixed_groups = sorted_mixed_df.group.unique()

In [99]:
list_mixed_groups

array([ 615, 1637, 5866, ..., 6173, 4325, 2816])

In [100]:
tops = list_mixed_groups[:200]

In [115]:
df_mixed[df_mixed.group.isin(tops)].groupby(['group', 'label']).count()[30:70]

pos_x  pos_y  pos_z  time
group label                           
554   0       6884   6884   6884  6884
      1       1312   1312   1312  1312
584   0       6792   6792   6792  6792
      1        621    621    621   621
607   0       6743   6743   6743  6743
      1        519    519    519   519
615   0       6791   6791   6791  6791
      1       1692   1692   1692  1692
639   0       6765   6765   6765  6765
      1        490    490    490   490
691   0       6708   6708   6708  6708
      1        524    524    524   524
716   0       6771   6771   6771  6771
      1        971    971    971   971
787   0       6672   6672   6672  6672
      1        576    576    576   576
797   0       6727   6727   6727  6727
      1        786    786    786   786
809   0       6947   6947   6947  6947
      1        591    591    591   591
810   0       6862   6862   6862  6862
      1        528    528    528   528
857   0       6714   6714   6714  6714
      1        592    592    592   592
941   0       6949   6949   6949  6949
      1        592    592    592   592
979   0       6990   6990   6990  6990
      1        848    848    848   848
988   0       6902   6902   6902  6902
      1        492    492    492   492
997   0       6818   6818   6818  6818
      1        517    517    517   517
1038  0       6701   6701   6701  6701
      1        684    684    684   684
1141  0       6694   6694   6694  6694
      1        558    558    558   558
1163  0       6770   6770   6770  6770
      1        643    643    643   643
1172  0       6820   6820   6820  6820
      1        487    487    487   487

In [97]:
df_mixed[df_mixed.group == 5686].groupby('label').count()

,pos_x,pos_y,pos_z,time,group
label,,,,,
0,6645,6645,6645,6645,6645
1,69,69,69,69,69


In [88]:
file_name = "group_5686.xyz" 
np.savetxt(WRITE_MIXED + file_name,
           df_mixed[df_mixed.group == 5686][['pos_y', 'pos_z', 'time']].values)

In [ ]:
5686 has 69 points only
1063 has 350 points 